In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
from torchvision import transforms, datasets
import torchvision
import numpy as np
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
transform = transforms.Compose(
    [transforms.Resize(224),transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [ ]:
trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True)

Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
trainset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [ ]:
testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False)

Files already downloaded and verified


In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
AlexNet = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


In [ ]:
AlexNet.to(device)
AlexNet.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
# For CIFAR-10 dataset, we have to set the number of classes to 10
AlexNet.classifier[6] = nn.Linear(4096,10)
AlexNet.eval()
AlexNet.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
optimiser = optim.Adam(params=AlexNet.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
EPOCHS = 10

for epoch in (range(EPOCHS)):
   
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        AlexNet.zero_grad()
        outputs = AlexNet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()
    print(loss)
    
print('Finished Training')

tensor(2.1131, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.6430, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.4257, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.5067, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.8690, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.9183, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.9182, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.2837, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.9044, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.7977, device='cuda:0', grad_fn=<NllLossBackward>)
Finished Training


In [ ]:
PATH = '/content/drive/My Drive/Alex/Weights/pytorch_model.pt'
torch.save(AlexNet, PATH)

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = AlexNet(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 63 %


In [ ]:
Alexnet_model = AlexNet

In [ ]:
new_classifier = nn.Sequential(*list(AlexNet.classifier.children())[:-2])
Alexnet_model.classifier = new_classifier

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True)

In [ ]:
X_train = []
y_train = []
for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        out = Alexnet_model(inputs)
        out = out.cpu()
        arr = out.detach().numpy()
        X_train.append(arr)
        y_train.append(labels)

In [ ]:
train1 = np.array(X_train)
shape = (50000,4096)
X_train1 = train1.reshape(shape)

train2 = np.array(y_train)
shape = (50000,1)
y_train1 = train2.reshape(shape)

In [ ]:
df1 = pd.DataFrame(X_train1)
df1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4056,4057,4058,4059,4060,4061,4062,4063,4064,4065,4066,4067,4068,4069,4070,4071,4072,4073,4074,4075,4076,4077,4078,4079,4080,4081,4082,4083,4084,4085,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,-22.357113,31.422415,-10.628951,-12.043396,-17.674187,-9.062549,-46.124676,-14.703256,-12.055867,-8.829786,-16.389862,-16.342442,-13.598350,-21.105852,-19.140966,-4.255147,-42.111500,-35.837383,-44.494629,-45.914837,-10.994503,-10.506296,-5.166290,-9.464074,-37.025398,-10.183127,-10.171567,-16.563847,-29.890295,-19.328579,-121.634369,-25.387840,-30.909454,-13.623099,-27.764534,-27.511850,-17.986017,-27.253393,-38.077553,-10.740028,...,-13.149470,-9.524014,-18.872583,-21.577198,-40.803162,-7.709839,-14.737472,-18.583851,-36.884705,-67.098038,-9.264766,-34.627281,-7.143264,-11.158209,-9.318055,-28.992920,-22.185459,-13.071280,-26.121201,-15.661840,-13.444955,-39.587082,-35.627716,-48.552654,-25.225731,-14.641625,-11.822296,-37.832901,-99.585587,-10.537926,-30.541100,-17.025642,-12.765079,-48.320259,-65.324738,-54.185482,-11.120714,-22.696390,-32.196693,-60.614437
1,-41.090469,-27.074329,-6.959046,-12.359156,-13.292556,-6.918788,-21.235939,-11.099258,-11.921501,-5.918112,-13.038670,-8.914562,-8.765764,-10.839570,-13.612573,-3.136043,-30.592226,20.384909,-17.816292,-41.452515,-8.390076,-4.497503,-3.613654,-28.166904,-23.401396,-5.711039,-8.108607,-15.710956,-22.407757,-14.815751,-10.397775,-16.454590,-25.752045,-7.370722,-25.702311,-20.751234,-14.972533,-16.239403,-36.086555,-5.256194,...,-7.255111,-8.581865,-11.659254,-13.739040,-20.824419,23.934469,-9.837081,-8.633455,-23.640789,-31.494579,-4.383939,-13.944918,-6.585386,-12.829565,-6.603324,-16.841436,-13.626602,-10.134998,-10.490880,-11.619777,-10.414021,-13.294508,-24.390261,-45.377663,-16.934086,-10.348657,-7.207860,-42.507458,-33.071621,-7.998910,-48.489407,-8.184752,-9.244842,-36.447227,6.850604,-55.623913,-7.203391,-10.322604,-22.404188,-32.614712
2,-24.384384,-21.194845,-6.978611,-11.151893,-14.915277,-3.467864,-33.904243,-9.035235,-6.986090,-7.618218,-12.046366,-7.550766,-15.064261,-15.363521,-13.418299,-2.486114,-24.535868,-36.558907,-21.055222,-31.029329,-7.584010,-4.501604,-3.586230,-16.123049,-27.657759,-6.933251,-7.971369,-14.653252,-11.383043,-12.534865,-60.450508,-21.062218,-31.862614,-10.453632,-17.149485,-17.109308,-12.435679,-14.914718,-35.614906,-7.694231,...,-10.596003,-7.873999,-11.503175,-10.622757,-23.386206,-51.154808,-6.323208,-13.249192,-14.896047,-28.920599,-9.145644,-21.461824,-7.077690,-11.821798,-5.780771,-15.870038,-16.233919,-9.531764,-17.733164,-9.533693,-13.110082,-17.433241,-25.548494,-32.806046,-2.375754,-10.217098,-5.370861,-41.188095,-12.599454,-11.036797,-18.847780,-11.139484,-8.983601,-21.075123,-26.575003,-37.656006,-7.471581,-12.061083,-27.756237,-38.131023
3,-60.991070,2.082626,-10.170972,-17.638025,-34.695778,-15.403118,-71.502296,-15.870802,-14.662900,-8.572796,-20.935797,-17.530512,-19.676445,-20.169817,-25.166666,-3.491534,-61.884899,10.431143,-36.502407,-62.128471,-15.131849,-15.116314,-2.617938,-66.440269,-48.807480,-9.997970,-12.201532,-22.253834,-64.520798,-21.839727,-44.512291,-36.097927,-55.421562,-14.327885,-26.882875,-32.588379,-29.103022,-26.643724,-51.827362,-9.559845,...,-12.600736,-12.571262,-16.165461,-19.549864,-59.586952,4.260221,-14.904366,-24.813843,-50.908386,-56.139900,-10.011593,-34.525459,-9.409728,-11.863654,-9.801031,-36.589325,-26.772860,-15.322851,-34.683899,-16.643763,-14.260525,-29.846741,-27.649597,-20.406931,-9.802055,-20.353527,-11.656566,-42.185455,-116.481323,-16.431320,-48.015533,-16.006651,-12.700379,-67.837929,-61.509384,-95.784203,-12.716416,-24.874596,-61.682240,-95.368759
4,-34.949051,1.722352,-10.326998,-8.819321,-9.636962,-8.047864,-26.300343,-7.342387,-8.811357,-7.382533,-8.201268,-8.979960,-9.398632,-12.149743,-15.749556,-3.958238,-2.898312,26.645174,-26.648420,-44.521133,-7.03326

In [ ]:
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                          shuffle=True)
X_test = []
y_test = []
for i, data in enumerate(testloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        out = Alexnet_model(inputs)
        out = out.cpu()
        arr = out.detach().numpy()
        X_test.append(arr)
        y_test.append(labels)

test1 = np.array(X_test)
shape = (10000,4096)
X_test1 = test1.reshape(shape)

test2 = np.array(y_test)
shape = (10000,1)
y_test1 = test2.reshape(shape)


In [ ]:
# Imports

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
X_train1.shape

(50000, 4096)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train1, train2)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [ ]:
y_pred = knn.predict(X_test1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier()

In [ ]:
clf.fit(X_train1,train2)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
## Random Forest Accuracy

correct = 0
total = 0

for i in range(0,10000):
    inputs, labels = X_test[i],test2[i]
    v = inputs.reshape(1,-1)
    outputs = clf.predict(v)
    total += 1
    if outputs == np.array(labels):
      correct +=1

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 69 %


In [ ]:
# KNN accuracy

correct = 0
total = 0

for i in range(0,10000):
    inputs, labels = X_test[i],test2[i]
    v = inputs.reshape(1,-1)
    outputs = knn.predict(v)
    total += 1
    if outputs == np.array(labels):
      correct +=1

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 68 %
